In [1]:
import traffic
from traffic.data import opensky
from traffic.core import Traffic
from rich.pretty import pprint
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import h5py
import os

In [2]:
def download_month(month: int, year: int, start_day = 1, cached=True):

    if month < 10:
        str_month = "0" + str(month)
    else:
        str_month = str(month)

    filename = "Frankfurt_LH_" + str(year)[2:] + str_month + ".h5"
    print(filename)
    next_month = month + 1 if month < 12 else  1
    str_next_month = str(next_month) if next_month > 9 else "0" + str(next_month)

    start_day_str = "0" + str(start_day) if start_day < 10 else str(start_day)
    datetime_str_start = str(year) + "-"+ str_month + "-" + start_day_str +  " 00:00:00"
    datetime_str_end = str(year) + "-" + str_next_month + "-01 00:00:00" if next_month != 1 else str(year+1) + "-" + str_next_month + "-01 00:00:00"
    datetime_current = datetime.strptime(datetime_str_start, '%Y-%m-%d %H:%M:%S')
    datetime_end = datetime.strptime(datetime_str_end, '%Y-%m-%d %H:%M:%S')

    while datetime_current < datetime_end:

        print(datetime_current)
        day = datetime_current.day
        if day < 10:
            day_str = "0" + str(day)
        else:
            day_str = str(day)

        h5_key = "LH_22" + str_month + day_str
        print(h5_key)
        str_current_day = str(datetime_current)
        str_next_day = str(datetime_current + timedelta(days = 1))
        print("getting data between "+str_current_day+ " and "+ str_next_day)

        trajectories = opensky.history(str_current_day,
                            stop= str_next_day,
                            arrival_airport ="EDDF",
                            cached=cached)
        try:
            callsigns_lh = trajectories.data.callsign.loc[trajectories.data.callsign.str.contains("DLH").astype(bool)]

            callsigns_lh = callsigns_lh.unique()
            callsigns_lh = callsigns_lh[~pd.isnull(callsigns_lh)]
            trajectories_lh = trajectories[callsigns_lh]
            print("add",h5_key,"to file")

            trajectories_lh.to_hdf(filename, key=h5_key, format = 'table')

        except AttributeError as e:
            print(e)

        datetime_current += timedelta(days = 1)


In [ ]:
download_month(11, 2022, 1, cached = False)

Output()

Frankfurt_LH_2211.h5
2022-11-01 00:00:00
LH_221101
getting data between 2022-11-01 00:00:00 and 2022-11-02 00:00:00


add LH_221101 to file


Output()

2022-11-02 00:00:00
LH_221102
getting data between 2022-11-02 00:00:00 and 2022-11-03 00:00:00


...

Output()

2022-11-11 00:00:00
LH_221111
getting data between 2022-11-11 00:00:00 and 2022-11-12 00:00:00


add LH_221111 to file


Output()

2022-11-12 00:00:00
LH_221112
getting data between 2022-11-12 00:00:00 and 2022-11-13 00:00:00


In [22]:
trajectories_lh.to_hdf(filename, key=h5_key, format = 'table')

In [3]:
with h5py.File("Frankfurt_LH_2210.h5", 'r') as f:
    print(list(f.keys()))

['LH_221001', 'LH_221002', 'LH_221003', 'LH_221004', 'LH_221005', 'LH_221006', 'LH_221007', 'LH_221008', 'LH_221009', 'LH_221010', 'LH_221011', 'LH_221012', 'LH_221013', 'LH_221014', 'LH_221015', 'LH_221016', 'LH_221017', 'LH_221018', 'LH_221019', 'LH_221020', 'LH_221021', 'LH_221022', 'LH_221023', 'LH_221024', 'LH_221025', 'LH_221026', 'LH_221027', 'LH_221028', 'LH_221029', 'LH_221030', 'LH_221031']
